In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
# import tensorflow_hub as hub
import nltk
import matplotlib.pyplot as plt
import tqdm
import transformers

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

2023-10-13 14:24:06.074797: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
[nltk_data] Downloading package punkt to /home/artem/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/artem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/artem/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# load data
df = pd.read_csv("sarcasm_data_clean_v1.csv")

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["headline_clean"], df["is_sarcastic"], test_size=0.3, random_state=42)
X_train.shape, X_test.shape

((19952,), (8551,))

In [4]:
# limit gpu memory (only relevant for very large models or if computing on local gpu)
# will insure that tensorflow only uses as much memory as is absolutely necessary

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


2023-10-13 14:25:12.478498: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-10-13 14:25:12.480324: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-10-13 14:25:12.507026: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-13 14:25:12.507882: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce MX150 computeCapability: 6.1
coreClock: 1.5315GHz coreCount: 3 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 44.76GiB/s
2023-10-13 14:25:12.508082: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-10-13 14:25:12.516675: I tensorflow/stream_executor/platform/defaul

In [5]:
# load tokenizer
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

# helper function to tokenize
def create_bert_input_features(tokenizer, docs, max_seq_length):

    all_ids, all_masks, all_segments= [], [], []
    for doc in tqdm.tqdm(docs, desc="Converting docs to features"):

        tokens = tokenizer.tokenize(doc)

        if len(tokens) > max_seq_length-2:
            tokens = tokens[0 : (max_seq_length-2)]
        # with newer versions of transformers you don't need to explicitely add CLS and SEP
        # they are automatically added by the tokenizer
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        ids = tokenizer.convert_tokens_to_ids(tokens)
        masks = [1] * len(ids) # [1,1,1.....] # < 500 ones

        # Zero-pad up to the sequence length.
        while len(ids) < max_seq_length:
            ids.append(0)
            masks.append(0)

        segments = [0] * max_seq_length # [0,0,0...] # 500 zeros
        all_ids.append(ids)
        all_masks.append(masks)
        all_segments.append(segments)

    encoded = np.array([all_ids, all_masks, all_segments])

    return encoded

In [6]:
# encode train and test data
train_features_ids, train_features_masks, train_features_segments = create_bert_input_features(tokenizer, X_train, max_seq_length=35)
test_features_ids, test_features_masks, test_features_segments = create_bert_input_features(tokenizer, X_test, max_seq_length=35)

Converting docs to features: 100%|██████████| 8551/8551 [00:02<00:00, 4131.89it/s]


In [7]:
MAX_SEQ_LENGTH = 35

inp_id = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_input_ids")
inp_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_input_masks")
inp_segment = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_segment_ids")
inputs = [inp_id, inp_mask, inp_segment]

hidden_state = transformers.TFBertModel.from_pretrained('bert-base-uncased')(inputs)
pooled_output = hidden_state[1]

dense1 = tf.keras.layers.Dense(128, activation='relu')(pooled_output)
drop1 = tf.keras.layers.Dropout(0.25)(dense1)
dense2 = tf.keras.layers.Dense(128, activation='relu')(drop1)
drop2 = tf.keras.layers.Dropout(0.25)(dense2)

output = tf.keras.layers.Dense(1, activation='sigmoid')(drop2)

model = tf.keras.Model(inputs=inputs, outputs=output)

model.compile(optimizer=tf.optimizers.Adam(learning_rate=2e-5,
                                           epsilon=1e-08),
              loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

2023-10-13 14:27:03.336678: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the 

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

In [8]:
# set bert model to untrainable
model.layers[3].trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
bert_input_ids (InputLayer)     [(None, 35)]         0                                            
__________________________________________________________________________________________________
bert_input_masks (InputLayer)   [(None, 35)]         0                                            
__________________________________________________________________________________________________
bert_segment_ids (InputLayer)   [(None, 35)]         0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 109482240   bert_input_ids[0][0]             
                                                                 bert_input_masks[0][0]       

In [10]:
# train model
EPOCHS=5
BATCH_SIZE=32

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2, verbose=1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath="./checkpoints/checkpoint_bert_v1_only_train-test-split", monitor="accuracy")

model.fit(x=[train_features_ids, train_features_masks, train_features_segments], y=y_train, 
          batch_size=BATCH_SIZE, epochs=EPOCHS, shuffle=True, callbacks=[early_stopping, checkpoint])

Epoch 1/5
624/624 [==============================] - 748s 1s/step - loss: 0.2834 - accuracy: 0.8838


2023-10-13 14:43:18.625127: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./checkpoints/checkpoint_bert_v1_only_train-test-split/assets


INFO:tensorflow:Assets written to: ./checkpoints/checkpoint_bert_v1_only_train-test-split/assets


Epoch 2/5
624/624 [==============================] - 750s 1s/step - loss: 0.1434 - accuracy: 0.9479


INFO:tensorflow:Assets written to: ./checkpoints/checkpoint_bert_v1_only_train-test-split/assets


INFO:tensorflow:Assets written to: ./checkpoints/checkpoint_bert_v1_only_train-test-split/assets


Epoch 3/5
  2/624 [..............................] - ETA: 12:28 - loss: 0.0596 - accuracy: 0.9844

KeyboardInterrupt: 

In [11]:
# test
predictions = model.predict([test_features_ids, test_features_masks, test_features_segments])
predictions = [1 if prob > 0.5 else 0 for prob in predictions]

print(classification_report(y_test, predictions))
pd.DataFrame(confusion_matrix(y_test, predictions))

              precision    recall  f1-score   support

           0       0.90      0.94      0.92      4551
           1       0.93      0.88      0.90      4000

    accuracy                           0.91      8551
   macro avg       0.91      0.91      0.91      8551
weighted avg       0.91      0.91      0.91      8551



,0,1
0,4266,285
1,471,3529
